## Product analytics
### Customer Segmentation in Python
 
### RFM-analysis

In [1]:
import pandas as pd
import numpy as np


# Matplotlib forms basis for visualization in Python
import matplotlib.pyplot as plt

# We will use the Seaborn library
import seaborn as sns
sns.set()

# Graphics in SVG format are more sharp and legible
get_ipython().run_line_magic('config', "InlineBackend.figure_format = 'svg'")

# Increase the default plot size and set the color scheme
plt.rcParams['figure.figsize'] = (8, 5)
plt.rcParams['image.cmap'] = 'viridis'

In [7]:
# Download dataset
orders = pd.read_csv('https://stepik.org/media/attachments/lesson/413464/RFM_ht_data.csv',sep=',')
orders.head(5)

/opt/tljh/user/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,InvoiceNo,CustomerCode,InvoiceDate,Amount
0,C0011810010001,19067290,2020-09-01,1716.00
1,C0011810010017,13233933,2020-09-01,1489.74
2,C0011810010020,99057968,2020-09-01,151.47
3,C0011810010021,80007276,2020-09-01,146.72
4,C0011810010024,13164076,2020-09-01,104.00


In [8]:
# Changing the data format in columns
orders['InvoiceDate'] = pd.to_datetime(orders['InvoiceDate'])
orders['InvoiceNo'] = orders['InvoiceNo'].apply(str)
orders['CustomerCode'] = orders['CustomerCode'].apply(str)

In [9]:
orders.shape

(332730, 4)

In [10]:
orders.dtypes

InvoiceNo               object
CustomerCode            object
InvoiceDate     datetime64[ns]
Amount                 float64
dtype: object

#### 1. What is the maximum number of purchases made by one user?

In [17]:
orders.groupby('CustomerCode').agg({'InvoiceNo' : 'count'}).sort_values('InvoiceNo', ascending = False).head(1)

,InvoiceNo
CustomerCode,
19057820,204


#### 2. RFM - Segmentation

In [18]:
orders['InvoiceDate'].describe()

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


count                  332730
unique                     30
top       2020-09-30 00:00:00
freq                    19781
first     2020-09-01 00:00:00
last      2020-09-30 00:00:00
Name: InvoiceDate, dtype: object

In [19]:
last_date = orders['InvoiceDate'].max()
last_date

Timestamp('2020-09-30 00:00:00')

In [20]:

rfmTable = orders.groupby('CustomerCode').agg({'InvoiceDate': lambda x: (last_date - x.max()).days, # Recency #Количество дней с последнего заказа
                                        'InvoiceNo': lambda x: len(x),      # Frequency #Количество заказов
                                        'Amount': lambda x: x.sum()}) # Monetary Value #Общая сумма по всем заказам

rfmTable['InvoiceDate'] = rfmTable['InvoiceDate'].astype(int)
rfmTable.rename(columns={'InvoiceDate': 'recency', 
                         'InvoiceNo': 'frequency', 
                         'Amount': 'monetary_value'}, inplace=True)


In [60]:
rfmTable.head()

,recency,frequency,monetary_value
CustomerCode,,,
02213019,19,1,1609.20
02213042,22,3,9685.48
02213071,29,1,415.00
02213088,23,1,305.00
02213092,25,1,1412.88


In [21]:
rfmTable.shape

(123733, 3)

In [23]:
# Check unique customers
orders['CustomerCode'].nunique()

123733

In [24]:
# Quantiles for segmentation
quantiles = rfmTable.quantile(q = [0.25, 0.5, 0.75])
quantiles

,recency,frequency,monetary_value
0.25,2.0,1.0,765.00
0.50,8.0,2.0,1834.48
0.75,16.0,3.0,4008.84


In [25]:
rfmSegmentation = rfmTable

In [26]:
# Function for segmentation
def RClass(value,parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 1
    elif value <= quantiles_table[parameter_name][0.50]:
        return 2
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 3
    else:
        return 4


def FMClass(value, parameter_name,quantiles_table):
    if value <= quantiles_table[parameter_name][0.25]:
        return 4
    elif value <= quantiles_table[parameter_name][0.50]:
        return 3
    elif value <= quantiles_table[parameter_name][0.75]: 
        return 2
    else:
        return 1


In [27]:
# RFM - Segmentation
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles))

rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles))

rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles))

rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str)                             + rfmSegmentation.F_Quartile.map(str)                             + rfmSegmentation.M_Quartile.map(str)

In [28]:
rfmSegmentation.head()

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
CustomerCode,,,,,,,
02213019,19,1,1609.20,4,4,3,443
02213042,22,3,9685.48,4,2,1,421
02213071,29,1,415.00,4,4,4,444
02213088,23,1,305.00,4,4,4,444
02213092,25,1,1412.88,4,4,3,443


#### 3. What is the upper limit for the amount of purchases from users with class 4 in subsegment M? (In other words: users whose purchase amount is from 0 to X fall into class 4 in the M subsegment)

In [29]:
rfmSegmentation.query('M_Quartile == 4').monetary_value.max()

765.0

#### 4. What is the lower bound on the number of purchases from users with class 1 in subsegment F?

In [30]:
rfmSegmentation.query('F_Quartile == 1').frequency.min()

4

#### 5. What is the maximum number of days that can elapse since the last purchase for a user to fall into class 2 in subsegment R?

In [31]:
rfmSegmentation.query('R_Quartile == 2').recency.max()

8

#### 6. How many users fell into segment 111?

In [45]:
rfmSegmentation['RFMClass'].value_counts().reset_index().query('index == "111"')

,index,RFMClass
1,111,9705


#### 7.How many users fell into segment 311?

In [47]:
rfmSegmentation = rfmSegmentation.reset_index()

In [48]:
rfmSegmentation.query('RFMClass == "311"').CustomerCode.count()

1609

#### 8. Which RFM segment has the largest number of users?
#### 9. Which RFM segment has the smallest number of users?
#### 10. How many users fell into the smallest segment?

In [96]:
rfmSegmentation['RFMClass'].value_counts().sort_values(ascending = False)

444    10624
111     9705
443     6729
344     6593
211     5847
       ...  
424       63
114       60
214       60
314       33
414        2
Name: RFMClass, Length: 64, dtype: int64

RFM "444" segment has the largest number of users.

RFM "414" segment has the smallest number of users.

2 users fell into the smallest segment.